In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ======================================================
# 1) 데이터 로딩
# ======================================================
crime_file = 'crime_rate_long.csv'
income_file = '2019년도 4월기준 14~18년 시군구별 평균소득월액.csv'

df_crime = pd.read_csv(crime_file, encoding='utf-8-sig')
df_income = pd.read_csv(income_file, encoding='cp949')

print("=" * 60)
print("✓ 데이터 로딩 완료")
print("=" * 60)
print("\n=== 범죄 데이터 샘플 ===")
print(df_crime.head())
print(f"\n레코드 수: {len(df_crime)}, 컬럼: {df_crime.columns.tolist()}")

print("\n=== 평균소득 데이터 샘플 ===")
print(df_income.head())
print(f"\n레코드 수: {len(df_income)}, 컬럼: {df_income.columns.tolist()}")

# ======================================================
# 2) 지역명 정규화 함수
# ======================================================
def normalize_region(text):
    """지역명을 '○○구' 형태로 통일"""
    if pd.isna(text):
        return None
    text = str(text).strip()
    text = text.replace('구청', '').replace('서울특별시', '').replace('서울시', '').replace('서울 ', '').strip()
    if not text.endswith('구'):
        text = text + '구'
    return text

# ======================================================
# 3) 범죄 데이터 전처리
# ======================================================
df_crime['지역'] = df_crime['region'].apply(normalize_region)
df_crime_clean = df_crime[['year', '지역', 'population', 'total', 'crime_rate']].copy()
df_crime_clean.columns = ['연도', '지역', '인구수', '범죄건수', '범죄율']

print("\n" + "=" * 60)
print("✓ 범죄 데이터 전처리 완료")
print("=" * 60)
print(df_crime_clean.head())
print(f"\n연도 범위: {df_crime_clean['연도'].min()} ~ {df_crime_clean['연도'].max()}")
print(f"지역 수: {df_crime_clean['지역'].nunique()}개")

# ======================================================
# 4) 평균소득 데이터 전처리
# ======================================================
df_income['지역'] = df_income['시군구'].apply(normalize_region)

# 연도 컬럼을 long format으로 변환
income_years = ['2014년', '2015년', '2016년', '2017년', '2018년']
df_income_long = pd.melt(
    df_income,
    id_vars=['지역'],
    value_vars=income_years,
    var_name='연도_원본',
    value_name='평균소득'
)

# 연도를 숫자로 변환 (2014년 -> 2014)
df_income_long['연도'] = df_income_long['연도_원본'].str.replace('년', '').astype(int)
df_income_long = df_income_long[['연도', '지역', '평균소득']].copy()

print("\n" + "=" * 60)
print("✓ 평균소득 데이터 전처리 완료")
print("=" * 60)
print(df_income_long.head())
print(f"\n연도 범위: {df_income_long['연도'].min()} ~ {df_income_long['연도'].max()}")
print(f"지역 수: {df_income_long['지역'].nunique()}개")

# ======================================================
# 5) 데이터 병합
# ======================================================
df_merged = df_crime_clean.merge(df_income_long, on=['연도', '지역'], how='inner')
df_merged = df_merged.dropna()

print("\n" + "=" * 60)
print("✓ 데이터 병합 완료")
print("=" * 60)
print(df_merged.head(10))
print(f"\n병합된 레코드 수: {len(df_merged)}개")
print(f"분석 연도: {df_merged['연도'].min()} ~ {df_merged['연도'].max()}")
print(f"분석 지역 수: {df_merged['지역'].nunique()}개")
print(f"분석 지역: {sorted(df_merged['지역'].unique())}")

# ======================================================
# 6) CSV 저장
# ======================================================
output_file = 'income_crime_merged.csv'
df_merged.to_csv(output_file, index=False, encoding='utf-8-sig')

print("\n" + "=" * 60)
print(f"✓ CSV 저장 완료: {output_file}")
print("=" * 60)
print("  컬럼: 연도, 지역, 인구수, 범죄건수, 범죄율, 평균소득")
print("\n데이터 통계:")
print(df_merged[['범죄율', '평균소득', '인구수']].describe())

print(f"\n✓ {output_file} 파일이 생성되었습니다.")

✓ 데이터 로딩 완료

=== 범죄 데이터 샘플 ===
  region  year  population  total  crime_rate
0    강남구  2014    237375.0   8851     3728.70
1    강동구  2014    185905.0   5392     2900.41
2    강북구  2014    141246.0   4030     2853.18
3    강서구  2014    234378.0   5289     2256.61
4    관악구  2014    248180.0   6781     2732.29

레코드 수: 264, 컬럼: ['region', 'year', 'population', 'total', 'crime_rate']

=== 평균소득 데이터 샘플 ===
         시군구    2014년    2015년    2016년    2017년    2018년
0  서울특별시 종로구  2723293  2755607  2737610  2870292  3007628
1   서울특별시 중구  2865653  2922120  3001390  3078491  3120706
2  서울특별시 용산구  2180357  2327359  2369072  2464669  2539369
3  서울특별시 성동구  2042650  2141397  2214343  2386608  2476470
4  서울특별시 광진구  1788230  1832328  1874423  1949150  1997491

레코드 수: 229, 컬럼: ['시군구', '2014년', '2015년', '2016년', '2017년', '2018년']

✓ 범죄 데이터 전처리 완료
     연도   지역       인구수  범죄건수      범죄율
0  2014  강남구  237375.0  8851  3728.70
1  2014  강동구  185905.0  5392  2900.41
2  2014  강북구  141246.0  4030  2853.18
3  2014  강서구